In [236]:
import numpy as np
import matplotlib.pyplot as pl
import time

In [237]:
from varname import nameof, argname
def copier(arg):
    og_name = argname('arg')
    new_name = og_name + '_copy'
    globals()[new_name] = arg

In [238]:
c = 3e8
G = 6.67e-11
pi = np.pi
const = 96/5*pi**(8/3)*(G/c**3)**(5/3)

Specific arguments

In [239]:
f0 = 120
Mc = 3e-4* 2e30
f_max = 200
T_obs = 1e2
beta = const*f0**(8/3)*Mc**(5/3)

f_signal = 40*f_max
f_ratio = 25
nt = round(f_signal*T_obs)
t = np.arange(nt)/f_signal

In [240]:
beta

2.7439303219997145e-07

Generating signal and adding noise

In [241]:
beta_arr = np.array([1e-7, 1e-8])

In [242]:
phi = -6*pi/5*f0*(1-8./3.*np.outer(t, beta_arr))**(5/8)/beta_arr
phi = np.mod(phi,2*pi)
signal = 1*np.exp(1j*phi)
nh = 50
noise = nh*np.random.normal(size = nt)
data = np.add(signal, noise[:,np.newaxis])

In [243]:
%reset_selective -f "^signal$"
%reset_selective -f noise
%reset_selective -f phi

In [244]:
def strobo(beta, data, f_ratio):
    new_t = -3/5*(1-8/3*beta*t)**(5/8)/beta
    f_new = f_signal/f_ratio
    
    new_t *= f_new #changes the frequency to the new downsampled frequency (approximately, not exactly)
    floor_t = np.floor(new_t) #For some reason floor works better than round. Take it to the nearest time index which are integers
    idx = np.nonzero(np.diff(floor_t)) #The step that downsamples
    resampled = data[idx]
    t_out = (new_t[idx]-new_t[0])/f_new
    return (resampled, t_out)

def strobo_parallel(beta, data, f_ratio):
    new_t = -3/5*(1-8/3*np.outer(t, beta))**(5/8)/beta
    f_new = f_signal/f_ratio
    
    new_t *= f_new #changes the frequency to the new downsampled frequency (approximately, not exactly)
    floor_t = np.floor(new_t) #For some reason floor works better than round. Take it to the nearest time index which are integers
    idx = np.nonzero(np.diff(floor_t)) #The step that downsamples
    resampled = data[idx]
    t_out = (new_t[idx]-new_t[0])/f_new
    return (resampled, t_out)

In [245]:
def height_calc_parallel(beta_arr):
    phi = -6*pi/5*f0*(1-8./3.*np.outer(t, beta_arr))**(5/8)/beta_arr
    phi = np.mod(phi,2*pi)
    signal = 1*np.exp(1j*phi)
    nh = 50
    noise = nh*np.random.normal(size = nt)
    data = np.add(signal, noise[:,np.newaxis])
    
    new_t = -3/5*(1-8/3*np.outer(t, beta))**(5/8)/beta
    out, t_out = strobo(beta, data, f_ratio)
    corrected = np.fft.fftshift(np.fft.fft(out))
    freq_corrected = np.fft.fftshift(np.fft.fftfreq(len(t_out), d=t_out[1]-t_out[0]))
    corrected = corrected[len(corrected)//2:]
    freq_corrected = freq_corrected[len(freq_corrected)//2:]
    nt_new = len(out)
    resampled_amplitudes = np.abs(corrected/nt_new)
    
    arg_max = np.argmax(resampled_amplitudes)
    peak_freq = freq_corrected[arg_max]
    peak_height = resampled_amplitudes[arg_max]
    return peak_freq, peak_height

In [246]:
phi = -6*pi/5*f0*(1-8./3.*np.outer(t, beta_arr))**(5/8)/beta_arr
phi = np.mod(phi,2*pi)
signal = 1*np.exp(1j*phi)
nh = 50
noise = nh*np.random.normal(size = nt)
data = np.add(signal, noise[:,np.newaxis])

In [247]:
new_t = -3/5*(1-8/3*np.outer(t, beta_arr))**(5/8)/beta_arr
f_new = f_signal/f_ratio

new_t *= f_new #changes the frequency to the new downsampled frequency (approximately, not exactly)
floor_t = np.floor(new_t) #For some reason floor works better than round. Take it to the nearest time index which are integers
diff = np.append(np.diff(floor_t, axis = 0), [[0,0]], axis = 0).T
idx = np.nonzero(diff)
resampled = data.T[idx]

In [257]:
len(idx[0])//2

31999

In [256]:
idx[0][len(idx[0])//2+1]

1

In [222]:
data[25,0]

(48.26144601581682+0.7071067060174357j)

In [220]:
data.T[idx][0]

(48.26144601581682+0.7071067060174357j)

In [ ]:

idx = np.argwhere(np.diff(floor_t, axis = 0)) #The step that downsamples
resampled = data[idx]
t_out = (new_t[idx]-new_t[0])/f_new

In [25]:
height_calc_parallel(beta_arr)

(149.38998219132637, array([2.31049608e-06, 1.96577743e-03]))